# greedy_decode

In [1]:
import torch

In [2]:
def greedy_decode(model, enc_out, start_token_id, max_len=50, enc_mask=None, device='cuda'):
    batch_size = enc_out.size(0)
    ys = torch.full((batch_size, 1), start_token_id, dtype=torch.long, device=device)
    
    for _ in range(max_len - 1):
        logits = model(ys, enc_out=enc_out, enc_mask=enc_mask)
        next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        ys = torch.cat([ys, next_token], dim=1)
    return ys


### ⚙️ Adım Adım Açıklama

### 1️⃣ Başlangıç Tokeni Oluşturma
```python

ys = torch.full((batch_size, 1), start_token_id, dtype=torch.long, device=device)
```


* Her örnek için decoder’ın başlayacağı token (<BOS> veya start_token_id) oluşturulur.

* batch_size kadar satır, 1 token uzunluğunda sütun.

### 2️⃣ Token Üretim Döngüsü

```python
for _ in range(max_len - 1):
```


* Maksimum max_len kadar token üretmek için döngü.

* -1, çünkü başlangıç tokeni zaten eklenmiş durumda.

### 3️⃣ Decoder’dan Logitleri Almak

```python
logits = model(ys, enc_out=enc_out, enc_mask=enc_mask)
```


* Decoder, mevcut ys dizisini kullanarak sonraki token olasılıklarını (logits) üretir.

* enc_out → Encoder çıktısı (cross-attention için)

* enc_mask → Encoder attention maskesi

### 4️⃣ En Yüksek Olasılığı Seçmek (Greedy)

```python
next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
```


* logits[:, -1, :] → Sadece en son üretilen tokenin olasılıkları alınır.

* torch.argmax → En yüksek olasılığa sahip token seçilir (tam Greedy).

### 5️⃣ Yeni Tokeni Mevcut Dizinin Sonuna Eklemek
```python
ys = torch.cat([ys, next_token], dim=1)
```


* Mevcut token dizisine yeni token eklenir.

* Döngü tekrar ettiğinde decoder yeni diziyi kullanır.

### 6️⃣ Sonuç

* Döngü tamamlandığında ys → tüm üretilmiş token dizilerini içerir.

* Greedy strateji her zaman en yüksek olasılığı seçer, bu yüzden deterministic ve hızlıdır.

## 💡 Not:

* Avantaj: Basit, hızlı, deterministic.

* Dezavantaj: “Daha iyi uzun cümleler” veya “yaratıcı” sonuç üretmekte zayıf olabilir.

* Daha yaratıcı/çeşitlilik için Top-K / Top-p sampling gerekir.

----

# top_k_sampling

In [3]:
import torch

@torch.no_grad()
def top_k_sampling(model, enc_out, enc_mask, start_token_id, max_len=50, k=10, device="cuda"):
    """
    Top-K Sampling ile token üretimi
    model      : Decoder modeli
    enc_out    : Encoder çıktısı (cross-attention için)
    enc_mask   : Encoder attention maskesi
    start_token_id : Başlangıç token ID
    max_len    : Maksimum üretilen token sayısı
    k          : Top-K değeri
    device     : "cuda" veya "cpu"
    """
    batch_size = enc_out.size(0)
    ys = torch.full((batch_size, 1), start_token_id, dtype=torch.long, device=device)

    for _ in range(max_len - 1):
        # Decoder’dan logitleri al
        logits = model(ys, enc_out=enc_out, enc_mask=enc_mask)
        
        # Son token logitleri
        logits_last = logits[:, -1, :]
        
        # Top-K maskesi uygula
        topk_values, topk_indices = torch.topk(logits_last, k=k, dim=-1)
        probs = torch.zeros_like(logits_last)
        probs.scatter_(-1, topk_indices, torch.softmax(topk_values, dim=-1))
        
        # Top-K üzerinden rastgele token seç
        next_token = torch.multinomial(probs, num_samples=1)
        
        # Yeni tokeni mevcut dizinin sonuna ekle
        ys = torch.cat([ys, next_token], dim=1)

    return ys


### ⚙️ Adım Adım Açıklama
#### 1️⃣ Başlangıç Tokeni Oluşturma
```python
ys = torch.full((batch_size, 1), start_token_id, dtype=torch.long, device=device)
```


* Decoder’ın başlayacağı token (<BOS> veya start_token_id) oluşturulur.

* Her örnek için bir satır, 1 token uzunluğunda sütun.

* Batch boyutu kadar token dizisi başlatılır.

### 2️⃣ Token Üretim Döngüsü
```python
for _ in range(max_len - 1):
```


* Maksimum max_len kadar token üretmek için döngü.

* -1 çünkü başlangıç tokeni zaten eklenmiş durumda.

#### 3️⃣ Decoder’dan Logitleri Almak
```python
logits = model(ys, enc_out=enc_out, enc_mask=enc_mask)
```


* Decoder mevcut ys dizisini kullanarak sonraki token olasılıklarını (logits) üretir.

* enc_out → Encoder çıktısı (cross-attention için).

* enc_mask → Encoder attention maskesi.

#### 4️⃣ Son Token Logitlerini Al
```python
logits_last = logits[:, -1, :]
```


* Sadece en son üretilen tokenin logitleri alınır.

* Önceki tokenler dikkate alınmaz, çünkü bu adımda bir sonraki token üretiliyor.

#### 5️⃣ Top-K Seçimi
```python
topk_values, topk_indices = torch.topk(logits_last, k=k, dim=-1)
```


* torch.topk ile en yüksek K olasılığa sahip tokenler seçilir.

* topk_values → olasılık değerleri

* topk_indices → token ID’leri

#### 6️⃣ Olasılıkları Normalize Et
```python
probs = torch.zeros_like(logits_last)
probs.scatter_(-1, topk_indices, torch.softmax(topk_values, dim=-1))
```


* Top-K tokenler dışındaki tüm tokenler sıfırlanır.

* torch.softmax ile seçilen K tokenin olasılıkları normalize edilir.

* scatter_ ile olasılık tensorüne yerleştirilir.

### 7️⃣ Rastgele Token Seçimi
```python
next_token = torch.multinomial(probs, num_samples=1)
```


* torch.multinomial ile olasılıklara göre rastgele token seçilir.

* Yani en yüksek olasılığa sahip token her zaman seçilmez, bu sayede çeşitlilik sağlanır.

#### 8️⃣ Tokeni Mevcut Dizinin Sonuna Ekle
```python
ys = torch.cat([ys, next_token], dim=1)
```

* Yeni token, mevcut token dizisinin sonuna eklenir.

* Döngü tekrar ettiğinde decoder, güncellenmiş diziyi kullanır.

#### 9️⃣ Döngü Sonrası

* Döngü tamamlandığında ys → tüm üretilmiş token dizilerini içerir.

* Top-K Sampling, yaratıcı ve çeşitli sonuçlar üretmek için deterministic olmayan bir yöntemdir.

💡 Notlar:

* k değeri küçüldükçe → daha deterministic (daha az çeşitlilik)

* k değeri büyüdükçe → daha yaratıcı ve çeşitli çıktı

* Greedy’ye göre daha fazla rastgelelik sağlar ama olasılık hesaplama biraz daha maliyetlidir.

----

# 🔹 Top-p Sampling

In [ ]:
def top_p_sampling(model, enc_out, enc_mask, start_token_id, max_len=20, p=0.9, device="cpu"):
    batch_size = enc_out.size(0)
    ys = torch.full((batch_size, 1), start_token_id, dtype=torch.long, device=device)

    for _ in range(max_len - 1):
        logits = model(ys, enc_out=enc_out, enc_mask=enc_mask)
        logits_last = logits[:, -1, :]  # Son token logitleri

        # Softmax ile olasılıkları hesapla
        probs = torch.softmax(logits_last, dim=-1)

        # Top-P maskesi oluştur
        sorted_probs, sorted_indices = torch.sort(probs, descending=True, dim=-1)
        cum_probs = torch.cumsum(sorted_probs, dim=-1)
        mask = cum_probs > p

        # İlk True değer dışındakileri kapat
        mask[:, 1:] = mask[:, :-1].clone()
        mask[:, 0] = False

        # P’nin dışındaki tokenleri sıfırla
        sorted_probs[mask] = 0.0

        # Normalize et
        sorted_probs = sorted_probs / sorted_probs.sum(dim=-1, keepdim=True)
        
        # Tokenleri geri yerleştir
        probs.zero_()
        probs.scatter_(-1, sorted_indices, sorted_probs)

        # Rastgele token seç
        next_token = torch.multinomial(probs, num_samples=1)

        # Tokeni mevcut dizinin sonuna ekle
        ys = torch.cat([ys, next_token], dim=1)

    return ys

### ⚙️ Adım Adım Açıklama (Markdown Formatı)
#### 1️⃣ Başlangıç Tokeni Oluşturma
```PYTHON
ys = torch.full((batch_size, 1), start_token_id, dtype=torch.long, device=device)
```


* Decoder’ın başlayacağı token (<BOS> veya start_token_id) oluşturulur.

* Her örnek için bir satır, 1 token uzunluğunda sütun.

* Batch boyutu kadar token dizisi başlatılır.

#### 2️⃣ Token Üretim Döngüsü
```python
for _ in range(max_len - 1):
```


* Maksimum max_len kadar token üretmek için döngü.

* -1 çünkü başlangıç tokeni zaten eklenmiş durumda.

#### 3️⃣ Decoder’dan Logitleri Almak
```python
logits = model(ys, enc_out=enc_out, enc_mask=enc_mask)
```


* Decoder mevcut ys dizisini kullanarak sonraki token olasılıklarını (logits) üretir.

* enc_out → Encoder çıktısı (cross-attention için).

* enc_mask → Encoder attention maskesi.

### 4️⃣ Son Token Logitlerini Al
```python
logits_last = logits[:, -1, :]
```


* Sadece en son üretilen tokenin logitleri alınır.

#### 5️⃣ Olasılıkları Hesapla
```python
probs = torch.softmax(logits_last, dim=-1)
```


* Logitleri olasılıklara dönüştürmek için softmax uygulanır.

#### 6️⃣ Top-P Maskesi Oluştur
```python
sorted_probs, sorted_indices = torch.sort(probs, descending=True, dim=-1)
cum_probs = torch.cumsum(sorted_probs, dim=-1)
mask = cum_probs > p
```


* Tokenleri olasılıklarına göre sırala (yüksekten düşüğe).

* Kümülatif olasılık (cum_probs) hesaplanır.

* Kümülatif olasılık p değerini aşan tokenler maskelenir.

#### 7️⃣ P Dışındaki Tokenleri Sıfırla
```python
mask[:, 1:] = mask[:, :-1].clone()
mask[:, 0] = False
sorted_probs[mask] = 0.0
```

* İlk tokenin dışındaki aşan kümülatif olasılıkları sıfırla.

* Bu şekilde olasılık dağılımı Nucleus (Top-P) içinde kalır.

#### 8️⃣ Normalize Et
```python
sorted_probs = sorted_probs / sorted_probs.sum(dim=-1, keepdim=True)
probs.zero_()
probs.scatter_(-1, sorted_indices, sorted_probs)
```


* Top-P içerisindeki token olasılıkları normalize edilir.

* Scatter ile orijinal token pozisyonlarına geri yerleştirilir.

#### 9️⃣ Rastgele Token Seçimi
```python
next_token = torch.multinomial(probs, num_samples=1)
```


* Softmax olasılıklarına göre rastgele token seçilir.

* Böylece, her zaman en yüksek olasılık seçilmez → yaratıcı ve çeşitli çıktılar sağlanır.

#### 10️⃣ Tokeni Mevcut Dizinin Sonuna Ekle
```python
ys = torch.cat([ys, next_token], dim=1)
```


* Yeni token, mevcut token dizisinin sonuna eklenir.

#### 11️⃣ Döngü Sonrası

* Döngü tamamlandığında ys → tüm üretilmiş token dizilerini içerir.

* Top-P Sampling, Top-K’den farklı olarak belirli bir olasılık kütlesini (p) kapsar.

💡 Notlar:

* p değeri küçüldükçe → daha deterministic, daha az çeşitlilik

* p değeri büyüdükçe → daha yaratıcı ve çeşitli çıktı

Greedy’ye göre çok daha rastgele ve yaratıcı sonuçlar sağlar, özellikle uzun metin üretiminde tercih edilir

---

# 🔹 Top-K ve Top-P Karşılaştırma Tablosu

| Özellik                 | Top-K Sampling                                                          | Top-P (Nucleus) Sampling                                                           |
| ----------------------- | ----------------------------------------------------------------------- | ---------------------------------------------------------------------------------- |
| **Seçim Kriteri**       | En yüksek K olasılığa sahip tokenler arasından rastgele seçim           | Kümülatif olasılığı `p`’yi aşmayan tokenler arasından rastgele seçim               |
| **Deterministik mi?**   | Hayır, rastgelelik mevcut                                               | Hayır, rastgelelik mevcut                                                          |
| **Kontrol Parametresi** | `k` (seçilecek en yüksek olasılıklı token sayısı)                       | `p` (toplam kümülatif olasılık eşiği, 0-1)                                         |
| **Avantaj**             | Basit, hızlı, çeşitlilik kolayca kontrol edilebilir                     | Olasılık dağılımına göre dinamik seçim → daha doğal ve yaratıcı metin üretimi      |
| **Dezavantaj**          | Sabit K nedeniyle bazen çok sıkıcı veya çok riskli tokenler seçilebilir | Hesaplama biraz daha maliyetli, p seçimi hassas                                    |
| **Yaratıcılık**         | Orta → K küçükse deterministic, büyükse çeşitlilik artar                | Yüksek → doğal ve çeşitli çıktılar sağlar, özellikle uzun metinlerde tercih edilir |
| **Tipik Kullanım**      | Kısa metin, basit cevaplar, hızlı inference                             | Uzun metin, yaratıcı üretim, doğal dil üretimi                                     |


---

# LLM ve transformer tabanlı modeller için hem Top-K hem Top-P Sampling’i destekleyen, esnek ve dinamik bir Python fonksiyon seti hazırlayalım. Kod, batch desteği ve GPU uyumluluğu içeriyor olsun. Ayrıca temperature parametresi ile sıcaklık kontrolü de ekleyelim; böylece daha yaratıcı veya deterministik sonuçlar elde edebiliriz.

----

In [5]:
import torch
import torch.nn.functional as F

def sample_logits(logits, top_k=None, top_p=None, temperature=1.0):
    """
    Logitlerden token seçimi için Top-K ve Top-P Sampling.

    Args:
        logits: [batch_size, vocab_size] tensor
        top_k: int, en yüksek K olasılığa sahip tokenler
        top_p: float, kümülatif olasılık eşiği (0-1)
        temperature: float, 1.0 varsayılan, <1 deterministik, >1 çeşitlilik

    Returns:
        next_token: [batch_size, 1] seçilen tokenler
    """
    if temperature != 1.0:
        logits = logits / temperature

    probs = F.softmax(logits, dim=-1)

    # Top-K Filtering
    if top_k is not None and top_k > 0:
        topk_values, topk_indices = torch.topk(probs, top_k, dim=-1)
        probs_filtered = torch.zeros_like(probs).scatter_(-1, topk_indices, topk_values)
        probs = probs_filtered

    # Top-P (Nucleus) Filtering
    if top_p is not None and 0.0 < top_p < 1.0:
        sorted_probs, sorted_indices = torch.sort(probs, descending=True, dim=-1)
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
        # Mask dışındaki tokenleri sıfırla
        sorted_mask = cumulative_probs > top_p
        # İlk maskeli token dahil edilmez
        sorted_mask[..., 1:] = sorted_mask[..., :-1].clone()
        sorted_mask[..., 0] = 0
        sorted_probs[sorted_mask] = 0.0
        # Tensorü orijinal sıraya geri döndür
        probs = torch.zeros_like(probs).scatter_(-1, sorted_indices, sorted_probs)

    # Normalize edelim
    probs = probs / probs.sum(dim=-1, keepdim=True)

    # Token seç
    next_token = torch.multinomial(probs, num_samples=1)
    return next_token


def generate_sequence(model, start_tokens, max_len=50, top_k=None, top_p=None, temperature=1.0, enc_out=None, enc_mask=None, device='cpu'):
    """
    LLM Decoder için esnek Top-K / Top-P / Temperature sampling üretimi.
    Args:
        model: decoder modeli
        start_tokens: [batch_size, 1] başlangıç tokenleri
        max_len: üretilecek token sayısı
        top_k: Top-K parametresi
        top_p: Top-P parametresi
        temperature: sıcaklık kontrolü
        enc_out: encoder çıktısı (cross-attention)
        enc_mask: encoder attention maskesi
        device: cpu/cuda

    Returns:
        ys: [batch_size, max_len] üretilmiş tokenler
    """
    ys = start_tokens.to(device)

    for _ in range(max_len - 1):
        logits = model(ys, enc_out=enc_out, enc_mask=enc_mask)
        logits_last = logits[:, -1, :]  # Sadece en son token
        next_token = sample_logits(logits_last, top_k=top_k, top_p=top_p, temperature=temperature)
        ys = torch.cat([ys, next_token], dim=1)

    return ys


## ⚡ Özellikler ve Avantajlar

* Hem Top-K hem Top-P

Aynı fonksiyon içinde seçilebilir.

* Temperature kontrolü

temperature<1 → deterministik, güvenli seçimler

temperature>1 → daha yaratıcı, çeşitlilik artar

* Batch desteği

Tek seferde tüm batch için token üretir.

* Encoder uyumlu

Cross-attention (enc_out ve enc_mask) parametreleri ile seq2seq modelleri için hazır.

* Dinamik seçim

Top-K veya Top-P tek başına veya birlikte kullanılabilir.

----

# 📍 Kodun Genel Yapısında “Search” Nerede Olur?
## 🔹 1. Model tanımı (Encoder–Decoder)

* Önce zaten yaptığımız gibi:

```bash
encoder = Encoder(...)
decoder = Decoder(...)
model = Seq2Seq(encoder, decoder)
```


* Bu kısımda modelin yapısı tanımlanır.
* Henüz herhangi bir “search” stratejisi uygulanmaz.

##🔹 2. Train Loop (Eğitim)

* Burada model, gerçek hedef dizilerini (“teacher forcing” ile) öğrenir.

```python
for epoch in range(num_epochs):
    for batch in train_loader:
        enc_out = encoder(src)
        outputs = decoder(trg_inp, enc_out)
        loss = criterion(outputs, trg_labels)
        ...
```


* Burada “search” YOK çünkü:

* Eğitim sırasında hedef token’lar zaten bilinir.
Model tahmin etmez, öğrenir.

## 🔹 3. Search / Generation (Inference veya Evaluation Aşaması)

* İşte bizim yazdığımız Top-K / Top-P / Greedy kodu burada devreye girer.

* Yani modeli eğittikten sonra:
```python

# ---- Inference / Generation ----
model.eval()

start_tokens = torch.tensor([[tokenizer.bos_token_id]]).to(device)

# LLM veya Transformer üretim aşaması
generated = generate_sequence(
    model=model.decoder,   # sadece decoder veya tam seq2seq
    start_tokens=start_tokens,
    max_len=50,
    top_k=50,              # veya None
    top_p=0.9,             # veya None
    temperature=0.8,
    enc_out=encoder_output, # encoder varsa
    enc_mask=encoder_mask,  # encoder maskesi
    device=device
)

decoded_text = tokenizer.decode(generated[0])
print(decoded_text)
```


Bu kısımda:

* Model artık kendi tahmin ettiği token’ları üretir.

* generate_sequence() içinde search stratejisi (greedy, top-k, top-p) uygulanır.

## 🔹 4. Metrics ve Evaluation

* Search sonrası elde ettiğin çıktı, burada ölçülür:
```python

bleu_score = calculate_bleu(pred_texts, ref_texts)
rouge = compute_rouge(pred_texts, ref_texts)

# 🔧 Özetle Dosya Akışı

| Aşama         | Kod Bölümü                     | Açıklama                            |
| ------------- | ------------------------------ | ----------------------------------- |
| 🔹 Model      | `encoder.py`, `decoder.py`     | Mimari tanımı                       |
| 🔹 Train Loop | `train_loop.py`                | Teacher forcing, loss hesaplama     |
| 🔹 Search     | `search.py` veya `generate.py` | **Greedy / Top-K / Top-P sampling** |
| 🔹 Metrics    | `metrics.py`                   | BLEU, ROUGE, Perplexity hesaplama   |


# 🧩 LLM / Seq2Seq Üretim Akış Diyagramı

```bash
                ┌────────────────────────┐
                │      DataLoader        │
                │  (tokenized inputs)    │
                └──────────┬─────────────┘
                           │
                           ▼
                ┌────────────────────────┐
                │        Encoder         │
                │  - Attention Layers    │
                │  - Positional Embeds   │
                └──────────┬─────────────┘
                           │
                 enc_out, enc_mask
                           │
                           ▼
                ┌────────────────────────┐
                │        Decoder         │
                │  - Self-Attention      │
                │  - Cross-Attention     │
                │  - FFN Layers          │
                └──────────┬─────────────┘
                           │
                           ▼
          ┌────────────────────────────────┐
          │        Search / Generation      │
          │--------------------------------│
          │  Greedy Sampling               │
          │  Top-K Sampling                │
          │  Top-P (Nucleus) Sampling      │
          └──────────┬─────────────────────┘
                     │
                     ▼
          ┌─────────────────────────────┐
          │        Postprocessing       │
          │  - Detokenize output        │
          │  - Trim <EOS> tokens        │
          └──────────┬──────────────────┘
                     │
                     ▼
          ┌─────────────────────────────┐
          │         Evaluation          │
          │  - BLEU / ROUGE / PPL       │
          │  - Accuracy (optional)      │
          └─────────────────────────────┘




# ⚙️ Akış Sırası (main.py’de)

```python
from model.encoder import Encoder
from model.decoder import Decoder
from model.seq2seq import Seq2Seq
from model.search import generate_sequence
from utils.tokenizer import tokenizer
from eval.metrics import calculate_bleu

# 1️⃣ Model oluştur
encoder = Encoder(...)
decoder = Decoder(...)
model = Seq2Seq(encoder, decoder).to(device)

# 2️⃣ Eğitim (train_loop.py)
train_model(model, train_loader, ...)

# 3️⃣ Inference (search.py)
enc_out, enc_mask = encoder(src)
generated_tokens = generate_sequence(
    model.decoder,
    enc_out=enc_out,
    enc_mask=enc_mask,
    start_token_id=tokenizer.bos_token_id,
    top_k=50, top_p=None,
    temperature=0.8,
    max_len=100
)

# 4️⃣ Çözümleme
decoded_text = tokenizer.decode(generated_tokens[0])
print("🧠 Model Çıktısı:", decoded_text)

# 5️⃣ Değerlendirme
score = calculate_bleu(decoded_text, reference_text)
print("🎯 BLEU Skoru:", score)
